In [1]:
import pandas as pd
df_rating = pd.read_csv('parsed_json/face-rating-data-2.csv', index_col = None)
df_guess_who_temp1 = pd.read_csv('parsed_json/guess-who-data-1.csv', index_col = None)
df_guess_who_temp2 = pd.read_csv('parsed_json/guess-who-data-2.csv', index_col = None)
df_guess_who1 = df_guess_who_temp1.dropna(subset=['question', 'eliminatedFaces'])
df_guess_who2 = df_guess_who_temp2.dropna(subset=['question', 'eliminatedFaces'])

In [2]:
print(df_guess_who1.info())
print(df_guess_who2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3130 entries, 3 to 4135
Data columns (total 5 columns):
allFaces            3130 non-null object
eliminatedFaces     3130 non-null object
question            3130 non-null object
questionResponse    3130 non-null object
uniqueID            3130 non-null int64
dtypes: int64(1), object(4)
memory usage: 146.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16014 entries, 0 to 21107
Data columns (total 5 columns):
allFaces            16014 non-null object
eliminatedFaces     16014 non-null object
question            16014 non-null object
questionResponse    16014 non-null object
uniqueID            16014 non-null int64
dtypes: int64(1), object(4)
memory usage: 750.7+ KB
None


In [4]:
#merge two guess who data sets together 
temp = pd.merge(df_guess_who1, df_guess_who2, how='outer', indicator='indicator_column') 
outfile = open('guesswho-data-together.csv','w')
temp.to_csv(outfile, index = False)
outfile.close() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19144 entries, 0 to 19143
Data columns (total 6 columns):
allFaces            19144 non-null object
eliminatedFaces     19144 non-null object
question            19144 non-null object
questionResponse    19144 non-null object
uniqueID            19144 non-null int64
indicator_column    19144 non-null object
dtypes: int64(1), object(5)
memory usage: 897.5+ KB


In [5]:
temp = pd.read_csv("guesswho-data-together.csv")
temp.head()

,allFaces,eliminatedFaces,question,questionResponse,uniqueID,indicator_column
0,"IMG_9891,IMG_0153,IMG_0446,IMG_9484,IMG_9846,I...","IMG_9891,IMG_0153,IMG_9846,IMG_8502,IMG_8198,I...",Is the person's hair long?,No,27947,left_only
1,"IMG_9078,IMG_8697,IMG_9574,IMG_7715,IMG_9906,I...","IMG_9078,IMG_8697,IMG_9574,IMG_7715,IMG_9906,I...",does the person have a ponytail?,No,4193,left_only
2,"IMG_7942,IMG_7806,IMG_9710,IMG_9048,IMG_0701,I...","IMG_7942,IMG_7806,IMG_9710,IMG_9048,IMG_0701,I...",am I able to see the word 'new' on his shirt f...,Yes,1140,left_only
3,"IMG_7517,IMG_8153,IMG_9861,IMG_7836,IMG_7988,I...","IMG_7517,IMG_8153,IMG_9861,IMG_7836,IMG_7988,I...",does her hair go past her shoulder,No,24339,left_only
4,"IMG_9755,IMG_0401,IMG_9927,IMG_0325,IMG_7851,I...","IMG_0401,IMG_9048,IMG_0641,IMG_0566,IMG_7806,I...",is the person male?,No,78278,left_only


In [6]:
df_rating.head()

,age,ethnicity,eye,faceID,gender,hair
0,20,african,brown,IMG_8622,male,black
1,24,white,brown,IMG_7942,male,black
2,25,african,black,IMG_0401,male,black
3,23,white,Blue,IMG_0018,male,Blonde
4,26,white,brown,IMG_0777,male,brown


In [4]:
#concatenate guess who and rating data set together in order in all_rows variable 
all_rows = []

unique_by_faceID = df_rating.drop_duplicates(subset=['faceID'], keep='first')
for qIndex, q in temp.iterrows():
    allFaces = q['allFaces'].split(',')
    for faceID in allFaces:
        eliminated = faceID in q['eliminatedFaces']
        faces = unique_by_faceID[unique_by_faceID['faceID'] == faceID]
        all_rows.append([
            faceID,
            q['question'],
            q['questionResponse'],
            eliminated,
            faces.iloc[0]['ethnicity'],
            faces.iloc[0]['eye'],
            faces.iloc[0]['gender'],
            faces.iloc[0]['hair'],
            faces.iloc[0]['age'],
            q['allFaces'],
            q['eliminatedFaces']
        ])


In [6]:
import re 
import csv
csv_file = open("temp-data.csv", "w")

# columns = ['faceId', 'age', 'ethnicity', 'gender', 'hair', 'questions', 'responses']
csv_writer = csv.writer(csv_file)
# csv_writer.writeheader()
csv_writer.writerow(['faceId', 'question', 'response', 'eliminated?',
                     'ethnicity', 'eyes', 'gender', 'hair', 'age',
                     'allFaces', 'eliminatedFaces'])
csv_writer.writerows(all_rows)

csv_file.close()

In [294]:
data = pd.read_csv("temp-data.csv")

In [261]:
import numpy as np

In [301]:
data = data.drop(columns=['allFaces', 'eliminatedFaces'])

In [302]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306304 entries, 0 to 306303
Data columns (total 9 columns):
faceId         306304 non-null object
question       306304 non-null object
response       306304 non-null object
eliminated?    306304 non-null bool
ethnicity      297673 non-null object
eyes           306304 non-null object
gender         306304 non-null object
hair           306304 non-null object
age            306304 non-null int64
dtypes: bool(1), int64(1), object(7)
memory usage: 19.0+ MB


In [305]:
import pandas as pd
pd.set_option('display.max_colwidth', 30)

In [307]:
data.sample(n=5)

,faceId,question,response,eliminated?,ethnicity,eyes,gender,hair,age
75385,IMG_0401,Does this person have very...,Yes,True,african,black,male,black,25
184945,IMG_0153,Is their skin white?,No,True,white,brown,female,brown,26
303516,IMG_8517,Does the person have on a ...,No,False,african,brown,male,black,20
17831,IMG_0295,does he have a beard,Yes,True,african,dark brown,female,black,22
203608,IMG_9153,Does the person have glasses?,Yes,True,white,green,female,brown,23


In [137]:
# df = pd.DataFrame(all_rows, columns=['faceId', 'question', 'response', 'eliminated?',
#                      'ethnicity', 'eyes', 'gender', 'hair','age', 'allFaces', 
#                      'eliminatedFaces'])
# df.head()

In [266]:
data['age'].describe()

count    306304.000000
mean         26.306810
std           6.135791
min          16.000000
25%          22.000000
50%          25.000000
75%          29.000000
max          61.000000
Name: age, dtype: float64

In [267]:
data['gender'].value_counts()

male      160509
female    142080
unsure      3715
Name: gender, dtype: int64

In [268]:
import re
def process_hair(): 
    hair = data["hair"]
    clean = []
    for line in hair:
        line = line.lower().strip()
        line = re.sub(r'\bbla\S*\b', "black", line)
        line = re.sub(r'\bbr\S*\b', "brown", line)
        if "blond" in line: 
            line = line.replace(line, "blond")
        if line == "dark": 
            line = line.replace(line, "black")
        if "medium brown" in line: 
            line = line.replace(line, "brown")
        if "grey brown" in line: 
            line = line.replace(line, "blond")
        if "dark brown" in line: 
            line = line.replace(line, "black")
        if "light brown" in line: 
            line = line.replace(line, "brown")
        clean.append(line)
    return clean


In [269]:
import re
def process_eyes(): 
    eyes = data["eyes"]
    clean = []
    for line in eyes:
        line = line.lower().strip()
        line = re.sub(r'\b\S*\wen\b', "green", line)
        if line == "grey": 
            line = line.replace(line, "blue")
        if line == "black": 
             line = line.replace(line, "dark brown")
        clean.append(line)
    return clean

In [270]:
word_list = process_hair()
check = {}
for line in word_list: 
    count = check.get(line, 0)
    count += 1
    check[line] = count

        
print(check)

{'blond': 27048, 'brown': 101068, 'black': 172799, 'red': 4357, 'blue': 1032}


In [271]:
word_list = process_eyes()
check = {}
for line in word_list: 
    count = check.get(line, 0)
    count += 1
    check[line] = count

        
print(check)

{'brown': 194750, 'blue': 25370, 'hazel': 5152, 'green': 15787, 'dark brown': 65245}


In [278]:
def age_groups(): 
    values = data['age']
    values = data["age"].astype(str).astype(int)
    
    ageList = []
    for age in values:
        if age > 15 and age <= 24:
            ageGroup = "15-24"
        if age > 24 and age <= 34:
            ageGroup = "25-34"
        if age > 34 and age <= 44: 
            ageGroup  = "35-44"
        if age > 44 and age <= 54: 
            ageGroup = "45-54"
        if age > 54 and age <= 64: 
            ageGroup = "55-64"
        if age > 64 :
            ageGroup = "65+"
        ageList.append(ageGroup)       
    return ageList

In [273]:
from collections import defaultdict
def count_age_groups():
    ages = data['age'].astype(str).astype(int)
    groups = defaultdict(int)
    for age in ages:
        if age > 15 and age <= 24:
            groups['15-24'] += 1
            ageGroup = "15-24"
        if age > 24 and age <= 34:
            groups['25-34'] += 1
        if age > 34 and age <= 44: 
            groups['34-44'] += 1
        if age > 44 and age <= 54: 
            groups['45-54'] += 1
        if age > 54 and age <= 64: 
            groups['54-64'] += 1
        if age > 64 :
            groups['65+'] += 1
    
    return groups

In [274]:
age_groups = count_age_groups()
age_groups

defaultdict(int,
            {'25-34': 145123,
             '45-54': 4153,
             '34-44': 18858,
             '15-24': 137070,
             '54-64': 1100})

In [279]:
word_list = age_groups()
check = {}
for line in word_list: 
    count = check.get(line, 0)
    count += 1
    check[line] = count

        
print(check)

{'25-34': 145123, '45-54': 4153, '35-44': 18858, '15-24': 137070, '55-64': 1100}


In [280]:
print(data['gender'].value_counts())
print(data.groupby('ethnicity')['gender'].value_counts())

male      160509
female    142080
unsure      3715
Name: gender, dtype: int64
ethnicity  gender
african    male      77762
           female    52567
           unsure     2590
latino     female    16543
           male       6573
white      male      76174
           female    64339
           unsure     1125
Name: gender, dtype: int64


In [281]:
#creating the target value: True Answer
def getTrueAnswer(): 
    question_answer = []
    index = 0
    for index in range(0, len(data["response"])):
        if data["eliminated?"][index] == True and data["response"][index] == "No":
            question_answer.append("yes")
        if data["eliminated?"][index] == False and data["response"][index] == "No":
            question_answer.append("no")
        if data["eliminated?"][index] == False and data["response"][index] == "Yes":
            question_answer.append("yes")    
        if data["eliminated?"][index] == True and data["response"][index] == "Yes":
            question_answer.append("no")
        index += 1
    return question_answer

In [282]:
hair = process_hair()
eyes = process_eyes()
age = age_groups()
true_answer = getTrueAnswer()

cleaning the questions 

In [283]:
#defining words to remove
#nltk stopwords
#the words that were kept: have, has, only, his, her. as they might be informative. 
stopwords = ['person', 'persons', 'in','to','i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "youre", "youve", "youll", "youd", 'your', 'yours', 'yourself', 'yourselves', 'it', "its", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "thatll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'because', 'as', 'until', 'while', 'into', 'through', 'during', 'off', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'most', 'other', 'some', 'such', 'no', 'nor', 'own', 'same', 'so', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "dont", 'should', "shouldve", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "arent", 'couldn', "couldnt", 'didn', "didnt", 'doesn', "doesnt", 'hadn', "hadnt", 'hasn', "hasnt", 'haven', "havent", 'isn', "isnt", 'ma', 'mightn', "mightnt", 'mustn', "mustnt", 'needn', "neednt", 'shan', "shant", 'shouldn', "shouldnt", 'wasn', "wasnt", 'weren', "werent", 'won', "wont", 'wouldn', "wouldnt"]

In [284]:
import re
#clean questions 
def cleanquestion(question):
    question = question.replace('[comma]', '').lower().strip()
    return re.sub(r'[^a-zA-Z0-9\s]', '', question, re.I|re.A)

#cleaning stopwords
#normalized_stopwords = np.unique([x.replace("'", '') for x in stopwords])

def removestopwords(sentence):
    newWords = []
    for word in sentence.split():
        if word not  in stopwords: 
            newWords.append(word)
    return ' '.join(newWords)


In [285]:
def clean_list(column):
    result = []
    for question in column:
        cleaned = cleanquestion(question)
        without_stop_words = removestopwords(cleaned)
#         print('question:%s' % question)
#         print('without_stop_words:%s' % without_stop_words)
        result.append(without_stop_words)
    return result

In [286]:
data['question'] = clean_list(data['question'])

In [287]:
data.head()

,faceId,question,response,eliminated?,ethnicity,eyes,gender,hair,age
0,IMG_9891,hair long,No,True,white,brown,female,blonde,30
1,IMG_0153,hair long,No,True,white,brown,female,brown,26
2,IMG_0446,hair long,No,False,white,brown,female,brown,49
3,IMG_9484,hair long,No,False,white,Blue,female,Blonde,43
4,IMG_9846,hair long,No,True,white,brown,male,brown,26


In [288]:
del data['eliminated?']
del data['response']
del data['hair']
del data['eyes']
del data['age']
var1 = pd.DataFrame({'hair': hair})
var2 = pd.DataFrame({'eyes': eyes})
var3 = pd.DataFrame({'age': age})
var4 = pd.DataFrame({'trueAnswer': true_answer})
varbs = [var1, var2, var3, var4]
varbs = pd.concat(varbs, axis = 1)
df = pd.concat([varbs, data], axis=1)

In [289]:
df.head()

,hair,eyes,age,trueAnswer,faceId,question,ethnicity,gender
0,blond,brown,25-34,yes,IMG_9891,hair long,white,female
1,brown,brown,25-34,yes,IMG_0153,hair long,white,female
2,brown,brown,45-54,no,IMG_0446,hair long,white,female
3,blond,blue,35-44,no,IMG_9484,hair long,white,female
4,brown,brown,25-34,yes,IMG_9846,hair long,white,male


In [290]:
outfile = open('clean-features.csv','w')
df.to_csv(outfile, index = False)
outfile.close()    

In [309]:
import pandas as pd
data = pd.read_csv("clean-features.csv")
data.info()
data.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306304 entries, 0 to 306303
Data columns (total 8 columns):
hair          306304 non-null object
eyes          306304 non-null object
age           306304 non-null object
trueAnswer    306304 non-null object
faceId        306304 non-null object
question      304928 non-null object
ethnicity     297673 non-null object
gender        306304 non-null object
dtypes: object(8)
memory usage: 18.7+ MB


,hair,eyes,age,trueAnswer,faceId,question,ethnicity,gender
0,blond,brown,25-34,yes,IMG_9891,hair long,white,female
1,brown,brown,25-34,yes,IMG_0153,hair long,white,female
2,brown,brown,45-54,no,IMG_0446,hair long,white,female
3,blond,blue,35-44,no,IMG_9484,hair long,white,female
4,brown,brown,25-34,yes,IMG_9846,hair long,white,male


In [311]:
data.sample(n=10)

,hair,eyes,age,trueAnswer,faceId,question,ethnicity,gender
140966,black,dark brown,25-34,no,IMG_0671,have pig face,white,female
12144,black,dark brown,25-34,no,IMG_9755,hair black,african,female
92628,brown,brown,25-34,yes,IMG_8889,woman,latino,female
110635,blond,blue,25-34,yes,IMG_0199,she have big nose,white,female
90896,black,brown,25-34,yes,IMG_9957,male,african,male
147244,black,brown,15-24,yes,IMG_9364,wear glasses,african,male
81715,brown,green,15-24,no,IMG_7562,wear glasses,white,female
221569,brown,blue,25-34,no,IMG_9695,old,latino,male
39021,brown,dark brown,15-24,yes,IMG_0746,wearing red glasses,african,female
85144,black,brown,25-34,yes,IMG_9379,female,white,female
